In [5]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train')

In [49]:
import pandas as pd

df = pd.DataFrame({
    'doc_id': list(map(lambda filename: '/'.join(filename.split('/')[-2:]), newsgroups_train.filenames)),
    'topic': list(map(lambda topic_id: newsgroups_train.target_names[topic_id], newsgroups_train.target)),
    'text': newsgroups_train.data,
})

df

,doc_id,topic,text
0,rec.autos/102994,rec.autos,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,comp.sys.mac.hardware/51861,comp.sys.mac.hardware,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,comp.sys.mac.hardware/51879,comp.sys.mac.hardware,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,comp.graphics/38242,comp.graphics,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,sci.space/60880,sci.space,From: jcm@head-cfa.harvard.edu (Jonathan McDow...
5,talk.politics.guns/54525,talk.politics.guns,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...
6,sci.med/58080,sci.med,From: bmdelane@quads.uchicago.edu (brian manni...
7,comp.sys.ibm.pc.hardware/60249,comp.sys.ibm.pc.hardware,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...
8,comp.os.ms-windows.misc/10008,comp.os.ms-windows.misc,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...
9,comp.sys.mac.hardware/50502,comp.sys.mac.hardware,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...
